In [ ]:
# Zhaoyang Xu, August 2021, code for technology policy


# Import necessary libraries
import codecs
import joblib
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpld3
import nltk
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import re
from __future__ import print_function
from sklearn import feature_extraction
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the dataset
df0 = pd.read_csv("policy1.csv")
df0.head()

# Define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.2)

# Fit the vectorizer to the 'clean' column of the dataframe and transform the data into a tf-idf matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df0["clean"])

# Extract the feature names (terms) from the tf-idf vectorizer
terms = tfidf_vectorizer.get_feature_names_out()

# Compute the pairwise cosine similarity distances and convert to a distance measure
dist = 1 - cosine_similarity(tfidf_matrix)

# Save the distance matrix to a CSV file
pd.DataFrame(dist).to_csv("distance_pairwise.csv")

In [ ]:
# Set the number of clusters for KMeans
num_clusters = 5

# Initialize and fit the KMeans clustering model on the tf-idf matrix
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

# Get the cluster labels for each document
clusters = km.labels_.tolist()

# Load a previously saved KMeans model
km = joblib.load('doc_cluster.pkl')

# Assign the cluster labels to the dataframe
df0["cluster"] = clusters

# Count the number of documents in each cluster
df0['cluster'].value_counts()

# Group the dataframe by cluster for aggregation purposes
grouped = df0['Complexity'].groupby(df0['cluster'])

# Calculate the mean complexity rank (1 to 100) per cluster
grouped.mean()

# Initialize MDS (Multidimensional Scaling)
MDS()

# Initialize MDS (Multidimensional Scaling) with 2 components
# The 'dissimilarity' parameter is set to 'precomputed' because we are using a precomputed distance matrix
# The 'random_state' parameter is set for reproducibility
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

# Fit the MDS model to the distance matrix and transform the data
# The resulting 'pos' array contains the 2D coordinates for each point (document)
pos = mds.fit_transform(dist)  # shape (n_samples, n_components)

# Extract the x and y coordinates from the transformed data
xs, ys = pos[:, 0], pos[:, 1]

# Print a message indicating that the top terms per cluster will be displayed
print("Top terms per cluster:")
print()

# Sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')

    # Print top n words per cluster
    for ind in order_centroids[i, :6]:  # replace 6 with desired number of words
        print(' %s' % terms[ind].split(' '))

In [ ]:
# Define cluster colors
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

# Set font for matplotlib
myfont = r'/System/LibraryFonts/Supplemental/Songti.ttc'
matplotlib.rcParams['font.family'] = ['Heiti TC']

# Define cluster names
cluster_names2 = {
    0: 'Enterprise, Recognize, High-tech enterprises, Service, Ten thousand yuan, Method', 
    1: 'Unit, Method, Country, Reward, Innovation, Review', 
    2: 'Innovation, Enterprise, Construction, Support, Service, Industry', 
    3: 'Project, Unit, Funds, Acceptance, Undertake, Plan', 
    4: 'Patent, Intellectual property, Enterprise, Funding, Protection, Unit'
}

# Create 'cityyear' column
df0["cityyear"] = df0.apply(lambda x: x["City"] + str(x["Year"]), axis=1)

# Display matplotlib plots inline
%matplotlib inline

# Create DataFrame with MDS results and cluster info
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=df0["cityyear"]))

# Group by cluster
groups = df.groupby('label')

# Create Plotly figure
fig = go.Figure()

# Add scatter plot traces for each cluster
for name, group in groups:
    fig.add_trace(go.Scatter(
        x=group.x, y=group.y,
        legendgroup=cluster_names2[name], name=cluster_names2[name],
        mode="markers", marker=dict(color=cluster_colors[name], size=10),
        text=group.title
    ))

# Update plot layout
fig.update_layout(
    autosize=False, width=1500, height=800,
    title='Clustering, Grouped by Topics'
)

# Show plot
fig.show()

# Save plot as HTML
fig.write_html("clustering_grouped_by_topic.html")
